In [ ]:
import re

import matplotlib.pyplot as plt
from commonroad.common.util import Interval
from commonroad.visualization.mp_renderer import MPRenderer

from OpenSCENARIO2CR.AbsRel import AbsRel
from OpenSCENARIO2CR.EsminiWrapper.EsminiWrapperProvider import EsminiWrapperProvider
from OpenSCENARIO2CR.EsminiWrapper.StoryBoardElement import EStoryBoardElementType
from OpenSCENARIO2CR.Osc2CrConverter import Osc2CrConverter, EFailureReason

In [ ]:
scenario_path = ""
run_viewer = True

In [ ]:
if run_viewer:
    wrapper = EsminiWrapperProvider().provide_esmini_wrapper()
    wrapper.grace_time = 1.0
    wrapper.max_time = 60.0
    wrapper.ignored_level = EStoryBoardElementType.ACT
    wrapper.view_scenario(scenario_path)

In [ ]:
# Setup Converter
converter = Osc2CrConverter(
    delta_t=0.1,
    source_file=scenario_path,
    grace_time=1.0,
    max_time=60.0,
    ignored_level=EStoryBoardElementType.ACT,
    log_to_console=False,
    log_to_file="/home/michael/log.txt",
    goal_state_time_step=AbsRel(interval=Interval(-5, 5), is_absolute=False),
    goal_state_position_length=50,
    goal_state_position_width=10,
    ego_filter=re.compile(r".*[Ee]go.*"),
    keep_ego_vehicle=True,
    do_run_cr_monitor=True,
    do_trim_scenario=True,
)

result = converter.run_conversion()
if isinstance(result, EFailureReason):
    print(f"Scenario failed due to {result}")

In [ ]:
# plot every second time_stamp of the scenario
if not isinstance(result, EFailureReason):
    scenario, planning_problem_set, statistics = result
    for i in range(0, max([obst.prediction.final_time_step for obst in scenario.dynamic_obstacles]) + 1, 2):
        plt.figure(figsize=(5, 5))

        plot_limit = None  # If non-null the renderer follows the ego vehicle
        if plot_limit is None:
            rnd = MPRenderer()
        else:
            rnd = MPRenderer(plot_limits=[-plot_limit, plot_limit, -plot_limit, plot_limit],
                             draw_params={"focus_obstacle_id": scenario.obstacles[0].obstacle_id})
        # plot the scenario at different time step
        scenario.draw(rnd, draw_params={'time_begin': i})
        # plot the planning problem set
        planning_problem_set.draw(rnd)
        rnd.render()